<a href="https://colab.research.google.com/github/yomyaykya/yomyay/blob/master/%D0%9F%D0%98%D0%A1lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.15.0

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.0`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
import numpy as np
from collections import Counter
from sklearn.datasets import fetch_20newsgroups
categories = ["comp.os.ms-windows.misc", "sci.crypt", "talk.religion.misc", "rec.autos" ]
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)


In [3]:
vocab = Counter()

In [4]:
for text in newsgroups_train.data:
    for word in text.split(' '):
        vocab[word.lower()] += 1


In [5]:
for text in newsgroups_test.data:
    for word in text.split(' '):
        vocab[word.lower()] += 1


In [6]:
total_words = len(vocab)
#сохранение индексов каждого слова
def get_word_2_index(vocab):
    word2index = {}
    for i, word in enumerate(vocab):
        word2index[word.lower()] = i
    return word2index


In [7]:
word2index = get_word_2_index(vocab) 
def get_batch(df, i, batch_size):
    batches = []
    results = []
    texts = df.data[i * batch_size:i * batch_size + batch_size]
    categories = df.target[i * batch_size:i * batch_size + batch_size]
    for text in texts:
        layer = np.zeros(total_words, dtype=float)
        for word in text.split(' '):
            layer[word2index[word.lower()]] += 1
        batches.append(layer)
    for category in categories:
        y = np.zeros((5), dtype=float)
        if category == 0:
            y[0] = 1.
        elif category == 1:
            y[1] = 1.
        elif category == 2:
            y[2] = 1.
        elif category == 3:
            y[3] = 1.
        results.append(y)
    return np.array(batches), np.array(results)

In [8]:
# Параметры обучения
learning_rate = 0.01
training_epochs = 15
batch_size = 150
display_step = 1

# Network Parameters
n_hidden_1 = 200 # скрытый слой
n_hidden_2 = 150 # скрытый слой
n_hidden_3 = 100 # скрытый слой

n_input = total_words # количество уникальных слов в текстах
n_classes = 5

input_tensor = tf.placeholder(tf.float32,[None, n_input],name="input")
output_tensor = tf.placeholder(tf.float32,[None, n_classes],name="output")
def multilayer_perceptron(input_tensor, weights, biases):
    # скрытый слой
    layer_1_multiplication = tf.matmul(input_tensor, weights['h1'])
    layer_1_addition = tf.add(layer_1_multiplication, biases['b1'])
    layer_1 = tf.nn.relu(layer_1_addition)
    
    # скрытый слой
    layer_2_multiplication = tf.matmul(layer_1, weights['h2'])
    layer_2_addition = tf.add(layer_2_multiplication, biases['b2'])
    layer_2 = tf.nn.relu(layer_2_addition)
  
    # скрытый слой
    layer_3_multiplication = tf.matmul(layer_2, weights['h3'])
    layer_3_addition = tf.add(layer_3_multiplication, biases['b3'])
    layer_3 = tf.nn.relu(layer_3_addition)

    # выходной слой
    out_layer_multiplication = tf.matmul(layer_3, weights['out'])
    out_layer_addition = out_layer_multiplication + biases['out']
    
    return out_layer_addition

# инициализация параметров сети
weights = {
  'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
  'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
  'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
  'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
}
biases = {
  'b1': tf.Variable(tf.random_normal([n_hidden_1])),
  'b2': tf.Variable(tf.random_normal([n_hidden_2])),
  'b3': tf.Variable(tf.random_normal([n_hidden_3])),
  'out': tf.Variable(tf.random_normal([n_classes]))
}
# создание модели
prediction = multilayer_perceptron(input_tensor, weights, biases)

# Фукнция потерь
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=output_tensor))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
init = tf.global_variables_initializer()# Запуск
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(newsgroups_train.data)/batch_size)
        for i in range(total_batch):
            batch_x,batch_y = get_batch(newsgroups_train,i,batch_size) 
            c,_ = sess.run([loss,optimizer], feed_dict={input_tensor: batch_x,output_tensor:batch_y})
            avg_cost += c / total_batch
        print("Эпоха:", '%04d' % (epoch+1), "loss=", "{:.16f}".format(avg_cost))
    print("Обучение завершено!")
    
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(output_tensor, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    total_test_data = len(newsgroups_test.target)
    batch_x_test, batch_y_test = get_batch(newsgroups_test, 0, total_test_data)
    print("Точность:", accuracy.eval({input_tensor: batch_x_test, output_tensor: batch_y_test}))

Эпоха: 0001 loss= 12212.2378104073650320
Эпоха: 0002 loss= 2306.7938036237442248
Эпоха: 0003 loss= 1864.4551544189453125
Эпоха: 0004 loss= 2169.3459254673548458
Эпоха: 0005 loss= 1239.2397744315010186
Эпоха: 0006 loss= 128.6762567928859085
Эпоха: 0007 loss= 7.9239470022065301
Эпоха: 0008 loss= 0.0000000000000000
Эпоха: 0009 loss= 0.0000000000000000
Эпоха: 0010 loss= 0.0000000000000000
Эпоха: 0011 loss= 0.0000000000000000
Эпоха: 0012 loss= 0.0000000000000000
Эпоха: 0013 loss= 0.0000000000000000
Эпоха: 0014 loss= 0.0000000000000000
Эпоха: 0015 loss= 0.0000000000000000
Обучение завершено!
Точность: 0.6819763
